In [25]:
from docx import Document
import pandas as pd
import os
from pathlib import Path
import re

In [350]:
from datetime import datetime
from configparser import ConfigParser
import psycopg2

filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [9]:
inputdir = Path(os.path.expanduser('~')) / "sandbox"
#os.listdir(inputdir)
filename='IUCNGlobalEcosystemTypology_v2-0_prepublication.docx'

In [328]:
#profiles=Document(inputdir / filename)


In [329]:
profiles

In [330]:
map_msg = 'Thumbnail map for illustrative purposes only. Users should consult sources and Table S4.1 to ensure data quality meets needs of their application.'

In [334]:
profiles = Document('Keith_etal_EarthsEcosystems_Appendix S4_3rdrevisions_subm+DK.docx')
from docx.shared import Pt
for section in profiles.sections:
    h = section.header
    f = section.footer
    #print(section)
    f.is_linked_to_previous = False
            
    for p in h.paragraphs:
        if re.search("^Contributors",p.text):
            #print(p.text)
            run = f.add_paragraph().add_run()
            font = run.font
            font.size = Pt(8)
            run.text = map_msg
        #else:
         #   emptyp = f.paragraphs[0]
          #  emptyp.text = ''
profiles.save('demo-footnote-no-pages.docx')

In [336]:
profiles = Document('Keith_etal_EarthsEcosystems_Appendix S4_3rdrevisions_subm+DK.docx')
for k in range(19,len(profiles.paragraphs)):
    p=profiles.paragraphs[k]
    if re.search("^Distribution:",p.text):
        run = p.add_run()
        font = run.font
        font.size = Pt(8)
        run.text = map_msg
profiles.save('demo-append-distribution-section.docx')

Solution from https://stackoverflow.com/questions/56658872/add-page-number-using-python-docx

In [341]:
from docx.oxml import OxmlElement, ns
from docx.enum.text import WD_ALIGN_PARAGRAPH
def create_element(name):
    return OxmlElement(name)

def create_attribute(element, name, value):
    element.set(ns.qn(name), value)


def add_page_number(run):
    fldChar1 = create_element('w:fldChar')
    create_attribute(fldChar1, 'w:fldCharType', 'begin')

    instrText = create_element('w:instrText')
    create_attribute(instrText, 'xml:space', 'preserve')
    instrText.text = "PAGE"

    fldChar2 = create_element('w:fldChar')
    create_attribute(fldChar2, 'w:fldCharType', 'end')

    run._r.append(fldChar1)
    run._r.append(instrText)
    run._r.append(fldChar2)

profiles = Document('Keith_etal_EarthsEcosystems_Appendix S4_3rdrevisions_subm+DK.docx')
for section in profiles.sections:
    h = section.header
    f = section.footer   
    #print(section)
    f.is_linked_to_previous = False
            
    for p in h.paragraphs:
        if re.search("^Contributors",p.text):
            #print(p.text)
            f.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
            pagerun=f.paragraphs[0].add_run()
            add_page_number(pagerun)
            run = f.add_paragraph().add_run()
            font = run.font
            font.size = Pt(8)
            run.text = map_msg
            

        #else:
         #   emptyp = f.paragraphs[0]
          #  emptyp.text = ''
profiles.save('demo-footnote-with-page-numbers.docx')


In [321]:
len(f.paragraphs)

2

In [477]:
profiles = Document('Keith_etal_EarthsEcosystems_Appendix S4_3rdrevisions_subm+DK.docx')
len(profiles.paragraphs)

1430

In [482]:
contribs=list()
for section in profiles.sections:
    h = section.header
    for p in h.paragraphs:
        if re.search("^Contributors",p.text):
            m=p.text.split(':')
            contribs.append(m[1])

item=None
items=list()
nr=0
for k in range(19,len(profiles.paragraphs)):
    p=profiles.paragraphs[k].text
    if re.search("^[TMFS]+[0-9].[0-9]",p):
        if item is not None:
            items.append(item)
        m = p.split(" ")
        item={'Code':m[0],'Name':p,'References':list(),'Extra':list(),'Contributors':list()}
        addref=False
        authors=contribs[nr].split(',')
        for aut in authors:
            item['Contributors'].append(aut.strip(' '))
        nr=nr+1
    else:
        if item is not None:
            m=p.split(':')
            if m[0] in ('Ecosystem properties','Distribution','Biome','Ecological drivers'):
                #print(m[0])
                item[m[0]]=m[1].strip(" ")
            elif m[0] in ('References'):
                addref=True
            else:
                if addref:
                    item['References'].append(p)
                else:
                    item['Extra'].append(p)
items.append(item)
#items[1]

In [485]:
items[105]

{'Code': 'MT2.2',
 'Name': 'MT2.2 Large seabird and pinniped colonies',
 'References': ['Ellis JC (2005) Marine Birds on land: A review of plant biomass, species richness, and community composition in seabird colonies. Plant Ecology 181, 227–241. ',
  'Otero XL, De La Peña-Lastra S,  Pérez-Alberti A, Ferreira TO, Huerta-Diaz MA (2018) Seabird colonies as important global drivers in the nitrogen and phosphorus cycles. Nature Communications 9, 246.',
  'Riddick SN, Dragosits U, Blackall TD, Daunt F, Wanless S, Sutton MA (2012) The global distribution of ammonia emissions from seabird colonies. Atmospheric Environment 55, 319e327',
  'MT3. Anthropogenic shorelines biome',
  'Exmouth Marina, Devon, UK.',
  'Credit: Red Zeppelin on Unsplash',
  'The Anthropogenic shorelines biome is distributed globally where urbanised and industrial areas adjoin the coast, and includes some more remote structures such as artificial islands. It includes marine interfaces constructed from hard, smooth surfac

In [486]:
conn = psycopg2.connect(**params)
cur = conn.cursor()


In [488]:
qrystr_traits=   "INSERT INTO efg_ecological_traits(code,language,description,contributors,version,update) values %s ON CONFLICT ON CONSTRAINT efg_ecological_traits_pkey DO UPDATE SET description=EXCLUDED.description, contributors=EXCLUDED.contributors, update=CURRENT_TIMESTAMP(0)"
qrystr_drivers=   "INSERT INTO efg_key_ecological_drivers(code,language,description,contributors,version,update) values %s ON CONFLICT ON CONSTRAINT efg_key_ecological_drivers_pkey DO UPDATE SET description=EXCLUDED.description, contributors=EXCLUDED.contributors, update=CURRENT_TIMESTAMP(0)"
qrystr_dist=   "INSERT INTO efg_distribution(code,language,description,contributors,version,update) values %s ON CONFLICT ON CONSTRAINT efg_distribution_pkey DO UPDATE SET description=EXCLUDED.description, contributors=EXCLUDED.contributors, update=CURRENT_TIMESTAMP(0)"

affected_rows = 0
for item in items:
    if 'Ecosystem properties' in item.keys():
        values=tuple([item['Code'],'en',item['Ecosystem properties'],item['Contributors'],'v2.1',datetime.today().date()])
        cur.execute(qrystr_traits,(values,))
        affected_rows=affected_rows+cur.rowcount
    if 'Ecological drivers' in item.keys():
        values=tuple([item['Code'],'en',item['Ecological drivers'],item['Contributors'],'v2.1',datetime.today().date()])
        cur.execute(qrystr_drivers,(values,))
        affected_rows=affected_rows+cur.rowcount
    if 'Distribution' in item.keys():
        values=tuple([item['Code'],'en',item['Distribution'],item['Contributors'],'v2.1',datetime.today().date()])
        cur.execute(qrystr_dist,(values,))
        affected_rows=affected_rows+cur.rowcount
print(affected_rows)

330


In [489]:
qry=" ".join(("select code,LENGTH(t.description), LENGTH(k.description), LENGTH(d.description)",
"from efg_ecological_traits t",
"full join efg_key_ecological_drivers k",
"USING(code,version,language)",
"full join efg_distribution d",
"USING(code,version,language)",
"where version='v2.1' AND language='en' order by code;"))
cur.execute(qry)
updated_codes=cur.fetchall()

In [490]:
conn.commit()

In [491]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


In [492]:
updated_codes


[('F1.1', 1601, 863, 158),
 ('F1.2', 1593, 1148, 151),
 ('F1.3', 2014, 848, 119),
 ('F1.4', 1280, 726, 84),
 ('F1.5', 1381, 815, 85),
 ('F1.6', 1944, 1079, 100),
 ('F1.7', 1501, 967, 148),
 ('F2.1', 2223, 660, 58),
 ('F2.10', 1959, 718, 152),
 ('F2.2', 2246, 800, 79),
 ('F2.3', 2031, 1111, 135),
 ('F2.4', 1817, 1029, 129),
 ('F2.5', 1419, 955, 89),
 ('F2.6', 1395, 1265, 113),
 ('F2.7', 1962, 832, 85),
 ('F2.8', 1522, 881, 165),
 ('F2.9', 1848, 807, 179),
 ('F3.1', 1053, 1108, 245),
 ('F3.2', 1600, 1040, 121),
 ('F3.3', 1516, 788, 170),
 ('F3.4', 1514, 1032, 179),
 ('F3.5', 1143, 1080, 153),
 ('FM1.1', 2011, 1176, 85),
 ('FM1.2', 2079, 742, 65),
 ('FM1.3', 1437, 671, 324),
 ('M1.1', 1298, 754, 76),
 ('M1.10', 2378, 601, 147),
 ('M1.2', 1948, 807, 190),
 ('M1.3', 1990, 735, 108),
 ('M1.4', 1772, 942, 211),
 ('M1.5', 1771, 734, 185),
 ('M1.6', 2069, 902, 69),
 ('M1.7', 1542, 682, 58),
 ('M1.8', 1898, 904, 80),
 ('M1.9', 1016, 1119, 274),
 ('M2.1', 2320, 664, 70),
 ('M2.2', 1639, 908, 87),

In [206]:
p=profiles.paragraphs[1000]
pp=p._parent
pp.part.header_part

<bound method DocumentPart.header_part of <docx.parts.document.DocumentPart object at 0x11bc300d0>>

In [216]:
dir(p.part.document)
len( p.part.document.paragraphs)
#p.part.text

1430

In [273]:
section = profiles.sections[2]
for p in section.header.paragraphs:
    print(p.text)
for p in section.footer.paragraphs:
    print(p.text)
    

In [288]:
dir(section.footer.part)
#section.footer.part

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_blob',
 '_content_type',
 '_default_footer_xml',
 '_document_part',
 '_element',
 '_package',
 '_partname',
 '_rel_ref_count',
 '_rels',
 'after_unmarshal',
 'before_marshal',
 'blob',
 'content_type',
 'drop_rel',
 'element',
 'get_or_add_image',
 'get_style',
 'get_style_id',
 'load',
 'load_rel',
 'new',
 'new_pic_inline',
 'next_id',
 'package',
 'part',
 'part_related_by',
 'partname',
 'relate_to',
 'related_parts',
 'rels',
 'target_ref']

In [300]:
for p in section.first_page_footer.paragraphs:
        print(p.text)

In [305]:
p.part.content_type

'application/vnd.openxmlformats-officedocument.wordprocessingml.footer+xml'